## Stokes Flow in Periodic Channel with Robin boundary.

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "none"#"last_expr_or_assign"

import numpy as np
import matplotlib.pyplot as plt
from util.logger import EventTracker
from util.basis_scaled import *
from util.plot_tools import *
from boundary_solvers.gauss_grid_2d import StokesDirichletProblem
from scipy.io import loadmat
import matplotlib.pyplot as plt
from util.interp import PiecewiseInterp2D
from hmm.stokes import *
from itertools import product
from hmm.stokes_deep import DeepMicroSolver, get_net
import matplotlib
import torch.nn as nn

#matplotlib.rcParams['text.usetex'] = True
#pretty_pyplot_layout()

np.random.seed(0)
net_dir = "/mnt/data0/emastr/article_training_nodecay/"
MESH_PATH = "/home/emastr/deep-micro-slip-model/data/mesh/"
figures_dir = "/home/emastr/deep-micro-slip-model/data/figures/"
simulation_dir = "/home/emastr/deep-micro-slip-model/data/stokes_fenics/"

## Multiscale Problem

We define a multiscale problem as follows. The PDE is a non-slip zero forcing Stokes flow Problem

\begin{equation}
\Delta u  - \nabla p = 0, \qquad \qquad \nabla \cdot u = 0   \qquad \text{inside}\quad \Omega
\end{equation}

with the boundary conditions

\begin{equation}
u = g,\qquad \text{on}\quad\partial\Omega
\end{equation}

the boundary is a two-dimensional pipe with corners at (0,-1) and (1,1), and a micro scale boundary:

## Coupling to Micro Domain

To couple the macro domain to the microscopic domain, we make use of the HMM framework.
We construct a set of evenly spaced micro problems based on the Stokes data. The macro problem is constructed with a pre-specified resolution in the x- and y-directions.

In [6]:
from dataclasses import dataclass
from multiprocessing import Pool

@dataclass
class HyperParams:
    nMic: int
    n_refine: int
    xDim: int
    yDim: int
    width: float
    height: float
    tol: float
    max_iter: int
        
    
    
class Setup():
    def __init__(self, data: StokesData, param: HyperParams, net: nn.Module, net_settings: dict):
        """Create a new HMM setup with the given parameters."""
        self.params = param
        self.data = data
        
        # Macro problem
        macro = StokesMacProb(data, lambda x,a: fft_interp(x, a, dom=data.dom[0]))
        macro_solver = MacroSolver(param.xDim, param.yDim, tol=param.tol)
        
        # Micro problems
        self.xPos = np.linspace(data.dom[0][0], data.dom[0][1], param.nMic+1)[1:] - param.width/2 - (data.dom[0][1]-data.dom[0][0]) / param.nMic/1
        micros = [StokesMicProb(data, x, param.width, param.height, 0.0, 8, 5, 5, n_refine=param.n_refine) for x in self.xPos]
        micro_solvers = [MicroSolver(m, tol=param.tol) for m in micros]
        deep_micro_solvers = [DeepMicroSolver(m, net, net_settings, tol=param.tol) for m in micros]
        
        # Convergence checker
        conv_checker = ConvergenceChecker(macro, micros, tol=param.tol)
        
        # HMM problem
        self.hmm_problem = StokesHMMProblem(macro, micros, data, convergence_checker=conv_checker)
        self.hmm_solver = IterativeHMMSolver(macro_solver, micro_solvers)
        self.hmm_deep_solver = IterativeHMMSolver(macro_solver, deep_micro_solvers)
        
        
    def solve(self, **kwargs):
        macro_guess = self.hmm_solver.macro_solver.solve(self.hmm_problem.macro)
        (macro_sol, micro_sols) = self.hmm_solver.solve(self.hmm_problem, macro_guess=macro_guess, verbose=True, maxiter=self.params.max_iter, tol=self.params.tol)
        (deep_macro_sol, deep_micro_sols) = self.hmm_deep_solver.solve(self.hmm_problem, macro_guess=macro_guess, verbose=True, maxiter=self.params.max_iter, tol=self.params.tol)
        return micro_sols, macro_sol, deep_micro_sols, deep_macro_sol, macro_guess
    
    
    def save_sol(self, macro_sol, deep_macro_sol, filename):
        data = self.params.__dict__
        data['sol_u'] = macro_sol.u.eval_grid()
        data['sol_v'] = macro_sol.v.eval_grid()
        data['deep_sol_u'] = deep_macro_sol.u.eval_grid()
        data['deep_sol_v'] = deep_macro_sol.v.eval_grid()
        print(np.linalg.norm(data['sol_u'] - data['deep_sol_u'])/np.linalg.norm(data['sol_u']))
        np.save(filename, data)
        
        
    def load_sol(filename):
        data = np.load(filename, allow_pickle=True).flatten()[0]
        u = data.pop('sol_u')
        v = data.pop('sol_v')
        u_deep = data.pop('deep_sol_u')
        v_deep = data.pop('deep_sol_v')
        par = HyperParams(**data)
        return par, u, v, u_deep, v_deep

## Solving using HMM iterations

To solve the problem, we make use of a sequence of micro solvers, as well as a micro solver.

In [7]:
# range of data to use : 25 - 59
    
w_list = [1, 2, 3, 4, 5, 6, 7, 8]
n_list = [3, 5, 7, 9, 11, 13, 15, 17]
i_list = list(range(50, 60, 1))

args = [(i, n, w) for i, n, w in product(i_list, n_list, w_list)]

device = "cpu" # "cuda:0"
net, net_settings = get_net(f"/home/emastr/phd/data/runs/fnoskip_big_data_48_{40000}.Torch", 256, "cpu", "float")        

def solve_save(i, n, w):
    data, info = StokesData.load_from_matlab(f"/home/emastr/phd/data/reference_2/run_{i}.mat")
    data_big_domain = data
    dom = data.dom
    eps = info["eps"]
    print(f"i={i}, (n={n}, w={w}")
    params = HyperParams(nMic=n, n_refine=0, xDim=21, yDim=21, width=w*eps, height=w*eps*0.8, tol=1e-8, max_iter=30)
    setup = Setup(data, params, net, net_settings)
    _, macro_sol, _, deep_macro_sol, _ = setup.solve()
    setup.save_sol(macro_sol, deep_macro_sol, f"/home/emastr/phd/data/hmm_sols/sol_{i}_{n}_{w}.npy")

In [8]:
Pool(8).starmap(solve_save, args)

i=50, (n=3, w=1
i=50, (n=13, w=1i=51, (n=7, w=1
i=50, (n=17, w=5i=51, (n=11, w=5

i=50, (n=7, w=5

i=51, (n=17, w=1
i=52, (n=5, w=5


Convergence (not necessarily to solution) at step 10/30tep 3/30Step 11/30Step 4/30Step 12/30Step 8/30Step 0/30Step 5/30
Convergence (not necessarily to solution) at step 22/30
Convergence (not necessarily to solution) at step 10/30
Convergence (not necessarily to solution) at step 8/30
Convergence (not necessarily to solution) at step 11/30
0.0004740721580776805
i=52, (n=5, w=6
Convergence (not necessarily to solution) at step 11/30/30Step 20/30
0.0006063574799917575
Convergence (not necessarily to solution) at step 9/30
i=50, (n=7, w=6
Convergence (not necessarily to solution) at step 23/30
Convergence (not necessarily to solution) at step 9/30
0.00832146379174411
i=50, (n=3, w=2
Convergence (not necessarily to solution) at step 9/30
0.0007130243723062457
i=51, (n=11, w=6
Convergence (not necessarily to solution) at step 10/30
0.002319224016826713
i=52, (n=5, w=7
Convergence (not necessarily to solution) at step 20/30Step 9/30
Convergence (not necessarily to solution) at step 9/30
Con